# Constants, Utility Functions, and Data Importing

In [ ]:
# Constants

In [ ]:
# Utility Functions

In [ ]:
# Data Importing

## <font color="red">*Exercise 1*</font>

<font color="red">As this week's challenging questions asks, we'd like you to 
think how LLM can help your final project.  Try to use the OpenAI API to analyze 
a small-sized dataset (Remember to monitor API use on your OpenAI account!). The 
data could a sample from the dataset you prepare for the final project or some 
others. If it's going be a conventional task like classification, compare and 
see how it could beat(or being defeated) by other algorithms you've learned from 
previous weeks. If it's a special task that you cannot find a learned algorithm 
to compare with, evaluate its performance on your own and try if you can improve 
by changing hyperparameters(see [here](https://platform.openai.com/docs/api-reference/chat/create)), the prompt, etc.

## <font color="red">*Exercise 2*</font>

<font color="red">Fine-tune an LLM. You can either use the model (llama-2-7b) in 
the example code or find another open-source LLM. You may use datasets provided 
by HuggingFace or a dataset you collect from somewhere else (for your final project). 
If the task happens to be the same as in exercise 1, You can choose to compare the 
performance between the OpenAI LLM and your fine-tuned LLM. You can also choose to 
compare the performance between the vanilla and the fine-tuned LLM.

## <font color="red">*Exercise 3*</font>
<font color="red">Use LLM to generate some data and compare the differences between model-generated 
data and actual data. This exercise should not be a repetition of exercise 1. You should 
focus more on analyzing language nuances, qualitatively or quantitatively. You should also 
notice how the choice of LLM has possibly impacted the language it uses.

## <font color="red">*Exercise 4*</font>

<font color="red">Compare how LLMs change their performance with different 
shots on your task. If the evaluation criterion is quantifiable, such as 
classification with ground truth labels, plot and show how accuracy changes. 
If the evaluation criterion cannot be easily quantified, such as the clarity 
of explaining a concept, use your imagination to do some comparison (for 
example, you can ask another LLM to rate its peer :)) If you find close-sourced 
LLM APIs pricey and are unsatisfied with responses from small-sized open-sourced 
LLMs, you can try large-sized LLMs (such as 70B version Llama-2) with Petals 
(see [here](https://colab.research.google.com/drive/1uCphNY7gfAUkdDrTx21dZZwCOUDCMPw8?usp=sharing) and [here](https://colab.research.google.com/drive/1Ervk6HPNS6AYVr3xVdQnY5a-TjjmLCdQ)).

## <font color="red">*Exercise 5*</font>

<font color="red">Using Actor - Critical method to improve an LLM's performance 
on your task or doing some experiments langauge style learning (For example, 
you can investigate how LLMs perceive different groups of people would write 
their dating profiles. This may serve as an opportunity to explore how LLMs 
semantically embed social groups and assess their appropriateness.)